In [1]:
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [ ]:
# Starting URL (first page)
base_url = "https://yellowpages.az/Az/results/"
params = "?search-title=&search-by=brand&sort-by=asc"
# Company page URL pattern
url_pattern = re.compile(r"https://yellowpages\.az/Az/business/.+")

page_number = 1
# Company page links
def scrape_page(page_number):
    if page_number == 1:
        url = f"{base_url}{params}"
    else:
        url = f"{base_url}page/{page_number}/{params}"

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    all_links = soup.find_all('a', href=True)
    matching_links = [a['href'] for a in all_links if url_pattern.match(a['href'])]
    
    return matching_links

# Getting information from links
def scrape_business_info(business_url):
    response = requests.get(business_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    section_title = soup.find('div', class_='section-title')
    
    if section_title:
        head_title_text = section_title.find('h2').text.strip() if section_title.find('h2') else None
        about_title_text = section_title.find('i').text.strip() if section_title.find('i') else None
    else:
        head_title_text = about_title_text = None

    icon_divs = soup.find_all('div', class_='icon')
    address, phone, email = None, None, None
    for div in icon_divs:
        icon_tag = div.find('i')
        icon_text_div = div.find_next_sibling('div', class_='icon-text')
        if 'fa-map-marker' in icon_tag.get('class', []):
            address = icon_text_div.text.strip()
    
        elif 'fa-phone' in icon_tag.get('class', []):
            phone_nums = icon_text_div.find_all('p', class_='txt-tl')  
            if len(phone_nums) > 1:
                phone_list = [phone_num.get_text(strip=True) for phone_num in phone_nums]  
                phone = ', '.join(phone_list)
            else:
                phone = phone_nums[0].text.strip() if phone_nums else None
    
        elif 'fa-envelope' in icon_tag.get('class', []):
            email = icon_text_div.text.strip()
        
    return {
        'header': head_title_text,
        'about': about_title_text,
        'address': address,
        'phone': phone,
        'email': email 
    }

all_matching_links = []
all_business_info = []

# While there are some companies, take the link and go to next page
while True:
    links = scrape_page(page_number)
    if page_number == 101:
        break
    # if not links:
    #     break 
    all_matching_links.extend(links)
    page_number += 1

# Getting information from each link one by one
for business_link in all_matching_links:
    business_info = scrape_business_info(business_link)
    
    if business_info:
        print(business_info['header'])
        all_business_info.append(business_info)
    else:
        print("No specific info found!")

df = pd.DataFrame(all_business_info)

“9 Ay”
“A Agro” MMC
“A and J Services”
“A+A Security”
“AA Legal”
“AA Services”
“ABC Consulting” MMC
“ABS”, American Bureau of Shipping
“Aba Logistics” MMC
“Abak-Az Crowe” Ltd
“ABU Arena”
“ACC Co. Ltd”
“ACT Azerbaijan”
“ADO-G”, Construction, Industry & Tourism
“ADV.AZ”
“AE Arma-Elektropanc Electrical Mechanical Industry Engineering Contracting & Trade Inc.”
“AEF Hotel”
“Abshirin”
“Active Personnel”
“Adidas”
“AFB Bank”
“AKSA Cosmetics”
“Aeservis”
“Af Hotel Aqua Park”
“Afen Plaza”
“Agat Siks”
“Agralia” MMC
“Akbis” MMC
“Aki Inter” Ltd
“Akme International”
“Aksima” MMC
“Albatros”
“Aldetur”
“Aldo”
“Alfa Audit” MMC
“Alfa Travel”
“Aliyev & Co.”
“Alkan Ltd”
“All Star” MMC
“AllPrint”
“Allure”
“Alpha Coaching & Consulting”
“Alten Group”
“Altent”
“Altstadt Hotel”
“Altun Kitab” MMC
“Altun Saray”
“Amadeus”
“Amapola Exclusive Events”
“Amass Group”
“AQP MTK”
“Ambassade De La Beaute”
“Ambassador”
“Amber Hotel”
“Ampresto Mebel”
“Anatolia Hotel”
“Anglo-American Business Services & Attas Pr Alliance”
“Ans

In [ ]:
df.head()

In [ ]:
df.to_excel('web-scraping.xlsx', sheet_name='YellowPages', index=False)